# Preprocessing Phase

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm
import random
import time
import scipy

from tsfresh import extract_relevant_features
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters
from tsfresh import extract_features

import itertools

In [2]:
data_frame = pd.read_csv("train.csv")
musteri_id = data_frame["musteri"].values

## me-data
me_data = pd.read_csv("monthly_expenditures.csv")
me_data.sektor = me_data.sektor.astype(str)
me_data.tarih = me_data.tarih.astype(str)

last_6_mo = ["20190101", "20190201", "20190301", "20190401", "20190501", "20190601"]
last_3_mo = ["20190401", "20190501", "20190601"]
last_2_mo = ["20190501", "20190601"]

# me_data.loc[(me_data["islem_adedi"]<=0), "islem_adedi"] = 1
# me_data["aylik_toplam_tutar"] /= me_data["islem_adedi"]

wanted_sektors = list(me_data["sektor"].unique())

print("1. Extracting sektor-tarih columns")
## Extracting sektor-tarih columns
combos = pd.DataFrame({'{}_{}'.format(a, b):
me_data[(me_data["sektor"]==a) & (me_data["tarih"]==b)]["aylik_toplam_tutar"]
for a, b in itertools.product(wanted_sektors, last_6_mo)})
combos["musteri"] = me_data["musteri"]
combos = combos.groupby('musteri').agg('sum')
combos = combos.reindex(axis='index', labels=musteri_id)
combos.fillna(0, inplace=True)
##

print("2. Calculating last x month max expenses")
## Spending stats for last x months
me_data_for_maxcats = me_data.groupby(["musteri", "sektor", "tarih"]).sum().reset_index(level=['sektor', 'tarih'])
me_data_for_maxcats.tarih = me_data_for_maxcats.tarih.astype(str)

me_data_last_6 = me_data_for_maxcats.sort_values(['aylik_toplam_tutar'], ascending=[False]).groupby('musteri').first()
me_data_last_3 = me_data_for_maxcats[me_data_for_maxcats["tarih"].isin(last_3_mo)].sort_values(['aylik_toplam_tutar'], ascending=[False]).groupby('musteri').first()
me_data_last_2 = me_data_for_maxcats[me_data_for_maxcats["tarih"].isin(last_2_mo)].sort_values(['aylik_toplam_tutar'], ascending=[False]).groupby('musteri').first()


##
#important_max_expense_features = ["RESTORAN_CATER", "GIYIM_AKSESUAR", "KUYUMCU", "ELKT_ESYA_BILG", "DIJITAL_URUNLR", "HARCAMA_YOK"]
important_max_expense_features = wanted_sektors + ["HARCAMA_YOK"]
##

me_data_last_6.loc[-me_data_last_6["sektor"].isin(important_max_expense_features), "sektor"] = "DIGER"
me_data_last_3.loc[-me_data_last_3["sektor"].isin(important_max_expense_features), "sektor"] = "DIGER"
me_data_last_2.loc[-me_data_last_2["sektor"].isin(important_max_expense_features), "sektor"] = "DIGER"

me_data_last_6["max_cat_last6"] = me_data_last_6[["sektor"]]
me_data_last_3["max_cat_last3"] = me_data_last_6[["sektor"]]
me_data_last_2["max_cat_last2"] = me_data_last_2[["sektor"]]

me_data_last_6 = me_data_last_6[["max_cat_last6"]]
me_data_last_3 = me_data_last_3[["max_cat_last3"]]
me_data_last_2 = me_data_last_2[["max_cat_last2"]]
##

me_data_last_6 = me_data_last_6.reindex(axis='index', labels=musteri_id).fillna("HARCAMA_YOK")
me_data_last_3 = me_data_last_3.reindex(axis='index', labels=musteri_id).fillna("HARCAMA_YOK")
me_data_last_2 = me_data_last_2.reindex(axis='index', labels=musteri_id).fillna("HARCAMA_YOK")

print("3. Max and mean category expenses")
for sektor in wanted_sektors:
    column_list = [sektor+"_"+month for month in last_6_mo]
    combos[sektor+"_max"] = combos[column_list].max(axis=1)
    combos[sektor+"_idxmax"] = combos[column_list].idxmax(axis=1).apply(lambda x: int(x.split("_")[-1][5]))
    combos[sektor+"_mean"] = combos[column_list].mean(axis=1)
    
print("4. Total monthly expenses")
for month in last_6_mo:
    column_list = [sektor+"_"+month for sektor in wanted_sektors]
    combos["total_"+month] = combos[column_list].sum(axis=1)
    
print("5. Mean monthly expenses")
column_list = ["total_"+month for month in last_6_mo]
combos["monthly_mean"] = combos[column_list].mean(axis=1)

print("6. Nonzero month count")
column_list = ["total_"+month for month in last_6_mo]
combos["nonz_m_c"] = 0
for cl in column_list:
    combos["nonz_m_c"] += (combos[cl]>0).astype(int)


#data_frame['kidem_suresi'].mask(data_frame['kidem_suresi'] <= 0, data_frame['kidem_suresi'].median()[0])
data_frame.loc[(data_frame["kidem_suresi"]<=0), "kidem_suresi"] = 1

for column in ['is_durumu', 'egitim', 'meslek_grubu']:
    data_frame[column].fillna(data_frame[column].mode()[0], inplace=True)
    
    

data_frame.set_index("musteri", inplace=True, drop=True)
data_frame = pd.concat([data_frame, combos, me_data_last_2, me_data_last_3, me_data_last_6], axis=1, join="inner") ## Merge expense stats w/ basic info

data_frame.drop(["tarih"], axis=1, inplace=True)


Xs = data_frame


Xs.to_csv("train_me_features.csv")

1. Extracting sektor-tarih columns
2. Calculating last x month max expenses
3. Max and mean category expenses
4. Total monthly expenses
5. Mean monthly expenses
6. Nonzero month count


In [3]:
Xs

,yas,kidem_suresi,egitim,is_durumu,meslek_grubu,target,DIJITAL_URUNLR_20190101,DIJITAL_URUNLR_20190201,DIJITAL_URUNLR_20190301,DIJITAL_URUNLR_20190401,...,total_20190201,total_20190301,total_20190401,total_20190501,total_20190601,monthly_mean,nonz_m_c,max_cat_last2,max_cat_last3,max_cat_last6
musteri,,,,,,,,,,,,,,,,,,,,,
23b2476c8a,44.0,46.0,5f8ca0f83b,915be3a7a4,44af6626d6,0,0.0,0.0,0.0,0.0,...,940.0,1540.0,3770.0,1470.0,0.0,1358.333333,5,GIYIM_AKSESUAR,GIYIM_AKSESUAR,GIYIM_AKSESUAR
5a7e3a7122,39.0,194.0,01a4f2c96c,915be3a7a4,44af6626d6,0,0.0,0.0,0.0,70.0,...,70.0,70.0,90.0,1800.0,3380.0,916.666667,6,ELKT_ESYA_BILG,ELKT_ESYA_BILG,ELKT_ESYA_BILG
2ec95c7499,38.0,182.0,7e6640bfe0,d36d84f51a,44af6626d6,0,0.0,0.0,0.0,0.0,...,40.0,0.0,0.0,0.0,0.0,6.666667,1,HARCAMA_YOK,HARCAMA_YOK,GIYIM_AKSESUAR
54399dac33,34.0,101.0,5f8ca0f83b,991c4998fb,44af6626d6,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2700.0,0.0,450.000000,1,ELKT_ESYA_BILG,ELKT_ESYA_BILG,ELKT_ESYA_BILG
97a74b2e58,41.0,125.0,7bb291e291,242927d0f5,91ed9c1e18,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,670.000000,1,HARCAMA_YOK,HARCAMA_YOK,TURIZM_KONAKLM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b51ec62cf3,20.0,24.0,5f8ca0f83b,51be29729b,44af6626d6,0,0.0,0.0,0.0,0.0,...,140.0,1710.0,0.0,1480.0,1440.0,795.000000,4,BENZIN_YAKIT,BENZIN_YAKIT,BENZIN_YAKIT
6dc2be5b51,22.0,56.0,7e6640bfe0,d36d84f51a,44af6626d6,0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,1.666667,1,HARCAMA_YOK,HARCAMA_YOK,RESTORAN_CATER
a5e7b7632c,41.0,188.0,7e6640bfe0,ba7b390fc4,bfd9cb6270,0,0.0,0.0,0.0,0.0,...,310.0,1540.0,400.0,1530.0,1280.0,888.333333,6,ELKT_ESYA_BILG,ELKT_ESYA_BILG,ELKT_ESYA_BILG


# For Test Data

In [4]:
data_frame = pd.read_csv("test.csv")
musteri_id = data_frame["musteri"].values

## me-data
me_data = pd.read_csv("monthly_expenditures.csv")
me_data.sektor = me_data.sektor.astype(str)
me_data.tarih = me_data.tarih.astype(str)

last_6_mo = ["20190101", "20190201", "20190301", "20190401", "20190501", "20190601"]
last_3_mo = ["20190401", "20190501", "20190601"]
last_2_mo = ["20190501", "20190601"]

# me_data.loc[(me_data["islem_adedi"]<=0), "islem_adedi"] = 1
# me_data["aylik_toplam_tutar"] /= me_data["islem_adedi"]

wanted_sektors = list(me_data["sektor"].unique())

print("1. Extracting sektor-tarih columns")
## Extracting sektor-tarih columns
combos = pd.DataFrame({'{}_{}'.format(a, b):
me_data[(me_data["sektor"]==a) & (me_data["tarih"]==b)]["aylik_toplam_tutar"]
for a, b in itertools.product(wanted_sektors, last_6_mo)})
combos["musteri"] = me_data["musteri"]
combos = combos.groupby('musteri').agg('sum')
combos = combos.reindex(axis='index', labels=musteri_id)
combos.fillna(0, inplace=True)
##

print("2. Calculating last x month max expenses")
## Spending stats for last x months
me_data_for_maxcats = me_data.groupby(["musteri", "sektor", "tarih"]).sum().reset_index(level=['sektor', 'tarih'])
me_data_for_maxcats.tarih = me_data_for_maxcats.tarih.astype(str)

me_data_last_6 = me_data_for_maxcats.sort_values(['aylik_toplam_tutar'], ascending=[False]).groupby('musteri').first()
me_data_last_3 = me_data_for_maxcats[me_data_for_maxcats["tarih"].isin(last_3_mo)].sort_values(['aylik_toplam_tutar'], ascending=[False]).groupby('musteri').first()
me_data_last_2 = me_data_for_maxcats[me_data_for_maxcats["tarih"].isin(last_2_mo)].sort_values(['aylik_toplam_tutar'], ascending=[False]).groupby('musteri').first()


##
#important_max_expense_features = ["RESTORAN_CATER", "GIYIM_AKSESUAR", "KUYUMCU", "ELKT_ESYA_BILG", "DIJITAL_URUNLR", "HARCAMA_YOK"]
important_max_expense_features = wanted_sektors + ["HARCAMA_YOK"]
##

me_data_last_6.loc[-me_data_last_6["sektor"].isin(important_max_expense_features), "sektor"] = "DIGER"
me_data_last_3.loc[-me_data_last_3["sektor"].isin(important_max_expense_features), "sektor"] = "DIGER"
me_data_last_2.loc[-me_data_last_2["sektor"].isin(important_max_expense_features), "sektor"] = "DIGER"

me_data_last_6["max_cat_last6"] = me_data_last_6[["sektor"]]
me_data_last_3["max_cat_last3"] = me_data_last_6[["sektor"]]
me_data_last_2["max_cat_last2"] = me_data_last_2[["sektor"]]

me_data_last_6 = me_data_last_6[["max_cat_last6"]]
me_data_last_3 = me_data_last_3[["max_cat_last3"]]
me_data_last_2 = me_data_last_2[["max_cat_last2"]]
##

me_data_last_6 = me_data_last_6.reindex(axis='index', labels=musteri_id).fillna("HARCAMA_YOK")
me_data_last_3 = me_data_last_3.reindex(axis='index', labels=musteri_id).fillna("HARCAMA_YOK")
me_data_last_2 = me_data_last_2.reindex(axis='index', labels=musteri_id).fillna("HARCAMA_YOK")

print("3. Max and mean category expenses")
for sektor in wanted_sektors:
    column_list = [sektor+"_"+month for month in last_6_mo]
    combos[sektor+"_max"] = combos[column_list].max(axis=1)
    combos[sektor+"_idxmax"] = combos[column_list].idxmax(axis=1).apply(lambda x: int(x.split("_")[-1][5]))
    combos[sektor+"_mean"] = combos[column_list].mean(axis=1)
    
print("4. Total monthly expenses")
for month in last_6_mo:
    column_list = [sektor+"_"+month for sektor in wanted_sektors]
    combos["total_"+month] = combos[column_list].sum(axis=1)
    
print("5. Mean monthly expenses")
column_list = ["total_"+month for month in last_6_mo]
combos["monthly_mean"] = combos[column_list].mean(axis=1)

print("6. Nonzero month count")
column_list = ["total_"+month for month in last_6_mo]
combos["nonz_m_c"] = 0
for cl in column_list:
    combos["nonz_m_c"] += (combos[cl]>0).astype(int)


data_frame.loc[(data_frame["kidem_suresi"]<=0), "kidem_suresi"] = 1

for column in ['is_durumu', 'egitim', 'meslek_grubu']:
    data_frame[column].fillna(data_frame[column].mode()[0], inplace=True)
    
    

data_frame.set_index("musteri", inplace=True, drop=True)
data_frame = pd.concat([data_frame, combos, me_data_last_2, me_data_last_3, me_data_last_6], axis=1, join="inner") ## Merge expense stats w/ basic info

data_frame.drop(["tarih"], axis=1, inplace=True)


Xs = data_frame

Xs.to_csv("test_me_features.csv")

1. Extracting sektor-tarih columns
2. Calculating last x month max expenses
3. Max and mean category expenses
4. Total monthly expenses
5. Mean monthly expenses
6. Nonzero month count
